In [ ]:
import os
from PIL import Image
import numpy as np
from IPython.display import display
import h5py 
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout
import os as os
import time
from keras.datasets import cifar100
from keras import backend as K
from sklearn.model_selection import train_test_split

In [ ]:
def resize(image, label):
    img = tf.image.per_image_standardization(image)
    img = tf.image.resize(image, (227,227))
    return img, label

In [ ]:
(xtrain, ytrain), (xtest, ytest) = keras.datasets.cifar100.load_data()
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size = 0.1, shuffle = True)

169009152/169001437 [==============================] - 14s 0us/step


In [ ]:
# xval, yval = xtrain[:5000], ytrain[:5000]

In [ ]:
#creating small dataset using tensors, returns tf dataset representation  
dstrain = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
dstest = tf.data.Dataset.from_tensor_slices((xtest, ytest))
dsvalidation = tf.data.Dataset.from_tensor_slices((xval, yval))

In [ ]:
#getting size of the dataset for buffer in input pipeline
train_size = len(list(dstrain))
train_size = len(list(dstest))
validation_size = len(list(dsvalidation))

In [ ]:
# shuffeling and creating batch of dataset  
x_train = (dstrain.map(resize).shuffle(buffer_size=train_size).batch(batch_size=32, drop_remainder=True))
x_test = (dstest.map(resize).shuffle(buffer_size=train_size).batch(batch_size=32, drop_remainder=True))
x_validation = (dsvalidation.map(resize).shuffle(buffer_size=train_size).batch(batch_size=32, drop_remainder=True))

In [ ]:
#AlexNet model structure graph
model = keras.models.Sequential([
    
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3), name = 'conv1'),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),

    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="valid", name = 'conv2'),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="valid", name = 'conv3'),
    BatchNormalization(),
    
    Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="valid", name = 'conv4'),
    BatchNormalization(),
    
    Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="valid", name = 'conv5'),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Dropout(0.5),

    Flatten(),
    
    Dense(4096, activation='relu'),
    Dropout(0.3),
    
    Dense(4096, activation='relu'),
    Dropout(0.3),
    
    Dense(100, activation='softmax')
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 23, 23, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 9, 9, 384)         8

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing pretrained weights
weights_dict = np.load('/content/drive/MyDrive/bvlc_alexnet (1).npy', encoding='bytes', allow_pickle=True).item()
key = [number for number, name in weights_dict.items()]
value = [name for number, name in weights_dict.items()]

In [ ]:
for i in range(8):
  print(key[i],":",value[i][0].shape, "i = ", i)

fc6 : (9216, 4096) i =  0
fc7 : (4096, 4096) i =  1
fc8 : (4096, 1000) i =  2
conv3 : (3, 3, 256, 384) i =  3
conv2 : (5, 5, 48, 256) i =  4
conv1 : (11, 11, 3, 96) i =  5
conv5 : (3, 3, 192, 256) i =  6
conv4 : (3, 3, 192, 384) i =  7


In [ ]:
#setting weights on the layes compatible to the pretrained weights
model.get_layer("conv1").set_weights([value[5][0], value[5][1]])
model.get_layer("conv3").set_weights([value[3][0], value[3][1]])

In [ ]:
#fitting the model 
model.fit(x_train, epochs = 150, validation_data = x_validation, 
          validation_freq=1)

Epoch 1/80
1406/1406 [==============================] - 60s 43ms/step - loss: 4.8159 - accuracy: 0.0453 - val_loss: 3.7837 - val_accuracy: 0.1593
Epoch 2/80
1406/1406 [==============================] - 59s 42ms/step - loss: 4.0458 - accuracy: 0.0984 - val_loss: 3.5622 - val_accuracy: 0.1887
Epoch 3/80
1406/1406 [==============================] - 60s 43ms/step - loss: 3.7920 - accuracy: 0.1300 - val_loss: 3.4064 - val_accuracy: 0.2117
Epoch 4/80
1406/1406 [==============================] - 61s 43ms/step - loss: 3.5988 - accuracy: 0.1597 - val_loss: 3.2364 - val_accuracy: 0.2302
Epoch 5/80
1406/1406 [==============================] - 60s 43ms/step - loss: 3.4470 - accuracy: 0.1819 - val_loss: 3.1162 - val_accuracy: 0.2562
Epoch 6/80
1406/1406 [==============================] - 60s 43ms/step - loss: 3.3205 - accuracy: 0.2028 - val_loss: 3.0584 - val_accuracy: 0.2638
Epoch 7/80
1406/1406 [==============================] - 60s 43ms/step - loss: 3.1997 - accuracy: 0.2247 - val_loss: 2.9619 -

In [ ]:
# Evaluating the trained model on the test data
model.evaluate(x_test)

312/312 [==============================] - 5s 15ms/step - loss: 1.7726 - accuracy: 0.5378


[1.7725964784622192, 0.5377604365348816]